In [1]:
import cv2
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2
import numpy as np

In [2]:
def get_face_points(face_landmarks):
  # Save the landmark coordinates for  the eyes
  right_landmarks = [33, 160, 157, 133, 153, 144]
  right_eye_points = []
  for landmark in right_landmarks:
    right_eye_points.append([face_landmarks[landmark].x,
                            face_landmarks[landmark].y,
                            face_landmarks[landmark].z])

  left_landmarks = [362, 385, 388, 263, 373, 380]
  left_eye_points = []
  for landmark in left_landmarks:
    left_eye_points.append([face_landmarks[landmark].x,
                            face_landmarks[landmark].y,
                            face_landmarks[landmark].z])

  # Save the landmark coordinates for  the mouth
  mouth_landmarks = [78, 81, 13, 311, 308, 402, 14, 178]
  mouth_points = []
  for landmark in mouth_landmarks:
    mouth_points.append([face_landmarks[landmark].x,
                            face_landmarks[landmark].y,
                            face_landmarks[landmark].z])
  return right_eye_points, left_eye_points, mouth_points

In [3]:
def get_EARs(l_p, r_p):

  p2_p6 = np.linalg.norm(l_p[1]-l_p[5])
  p3_p5 = np.linalg.norm(l_p[2]-l_p[4])
  p1_p4 = np.linalg.norm(l_p[0]-l_p[3])
  left_EAR = (p2_p6 + p3_p5)/(2*p1_p4)

  p2_p6 = np.linalg.norm(r_p[1]-r_p[5])
  p3_p5 = np.linalg.norm(r_p[2]-r_p[4])
  p1_p4 = np.linalg.norm(r_p[0]-r_p[3])
  right_EAR = (p2_p6 + p3_p5)/(2*p1_p4)

  return left_EAR, right_EAR

In [4]:
def get_MAR(m_p):

  p2_p8 = np.linalg.norm(m_p[1]-m_p[7])
  p3_p7= np.linalg.norm(m_p[2]-m_p[6])
  p4_p6 = np.linalg.norm(m_p[3]-m_p[5])
  p1_p5 = np.linalg.norm(m_p[0]-m_p[4])
  MAR = (p2_p8 + p3_p7 + p4_p6)/(2*p1_p5)

  return MAR

In [6]:
cap = cv2.VideoCapture(0)

mouth_landmarks = [78, 81, 13, 311, 308, 402, 14, 178]
left_landmarks = [362, 385, 388, 263, 373, 380]
right_landmarks = [33, 160, 157, 133, 153, 144]
index_list = mouth_landmarks + left_landmarks + right_landmarks

BaseOptions = mp.tasks.BaseOptions
FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task'),
    running_mode=VisionRunningMode.IMAGE)
i = 0
with FaceLandmarker.create_from_options(options) as landmarker:
    while True:
        i += 1
        ret, frame = cap.read()
        if ret == False:
            break
        height, width, _ = frame.shape
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)

        results = landmarker.detect(mp_image)

        if results.face_landmarks is not None:
            face_landmarks_list = results.face_landmarks
            for idx in range(len(face_landmarks_list)):
                face_landmarks = face_landmarks_list[idx]
                face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
                face_landmarks_proto.landmark.extend([
                    landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks])
                for index in index_list:
                    x = int(face_landmarks[index].x * width)
                    y = int(face_landmarks[index].y * height)
                    cv2.circle(frame, (x, y), 2, (0, 255, 0), 1)
            
        if i % 7 == 0:
            try:
                right_eye_points, left_eye_points, mouth_points = get_face_points(results.face_landmarks[0])
            except:
                right_eye_points = right_eye_points
                left_eye_points = left_eye_points 
                mouth_points = mouth_points
            left_EAR, right_EAR = get_EARs(np.array(left_eye_points), np.array(right_eye_points))
            MAR = get_MAR(np.array(mouth_points))
        if i > 7:
            cv2.putText(frame,f"Left EAR: {left_EAR:.2f}",(30, 30),cv2.FONT_HERSHEY_DUPLEX,0.7,(0, 255, 255),1,)
            cv2.putText(frame,f"Right EAR: {right_EAR:.2f}",(30, 60),cv2.FONT_HERSHEY_DUPLEX,0.7,(0, 255, 255),1,)
            cv2.putText(frame,f"MAR: {MAR:.2f}",(30, 90),cv2.FONT_HERSHEY_DUPLEX,0.7,(0, 255, 255),1,)
        cv2.imshow("Frame", frame)
        k = cv2.waitKey(1) & 0xFF
        if k == 27:
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1702471389.107217    9315 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1702471389.111127    9473 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 21.2.6), renderer: Mesa Intel(R) UHD Graphics 620 (KBL GT2)
W0000 00:00:1702471389.112291    9315 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
